In [1]:
import os

In [2]:
%pwd

'e:\\Immediate watch\\September\\end to end DEEP learning Kindey project\\CodeME\\Kidney-Disease-Classification-MLFLOW-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Immediate watch\\September\\end to end DEEP learning Kindey project\\CodeME\\Kidney-Disease-Classification-MLFLOW-DVC'

In [10]:
from dataclasses import dataclass
from pathlib import Path
 
#  ENTITY
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self)->TrainingConfig: #get_training_config is used to get the training configuration
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        create_directories([Path(training.root_dir)])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.root_dir),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config

In [26]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self): #it is used to generate more images from the existing images
         #datagenerator is used to generate more images from the existing images

        datagenerator_kwargs = dict(
            rescale = 1./255 #images  are taken in range of 0-255 ,so we are rescaling it to 0-1
            ,validation_split=0.20 #20% of data is used for validation
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],#(224,224) image size ,except the channel
            batch_size=self.config.params_batch_size,# 16 images are taken at a time (spechify batch)
            interpolation="bilinear" # resize the image
        ) #it is used to load the data from the directory in this conditions

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs#Unpacking the dictionary ,cuz helpful when dealing with a large number of parameters.We don't need to write each parameter
        )  #image data generator is used to generate more images from the existing images

        self.valid_generator = valid_datagenerator.flow_from_directory(  #, flow_from_directory is used to load the images from the directory
            directory=self.config.training_data,#same directory as above
            subset="validation",#this is used for validation purpose 20%
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,#randomly rotate images in the range (degrees, 0 to 180) 
                horizontal_flip=True,#randomly flip images
                width_shift_range=0.2,#randomly translate images horizontally (fraction of total width)
                height_shift_range=0.2,#randomly translate images vertically (fraction of total height)
                shear_range=0.2,#set range for random shear
                zoom_range=0.2,
                **datagenerator_kwargs
            ) #it helps to your CNN model gneralize better by creating slightly modified versions of the same image.
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(#, flow_from_directory is used to load the images from the directory
            directory=self.config.training_data,#same directory as above
            subset="training",#this is used for training purpose 80%
            shuffle=True,#shuffle the data
            **dataflow_kwargs #unpacking the dictionary
        )

    
    @staticmethod #static method is used to define a method that doesn't operate on an instance of the class
    def save_model(path: Path, model: tf.keras.Model):
        model.save(f"{path}.h5")



    
    def train(self):
        #// (floor division) is used to get the integer value(remove decimal value)
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size #number of steps per epoch
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size#number of validation steps

        self.model.fit(#fit is used to train the model
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [30]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2025-10-04 09:06:43,369: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-04 09:06:43,379: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-04 09:06:43,385: INFO: common: created directory at: artifacts]
[2025-10-04 09:06:43,390: INFO: common: created directory at: artifacts\model_training]


Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 879s 38s/step - loss: 11.0643 - accuracy: 0.5955 - val_loss: 9.6432 - val_accuracy: 0.4375
[2025-10-04 09:21:32,907: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2025-10-04 09:21:36,698: INFO: builder_impl: Assets written to: artifacts\model_training\assets]
